In [1]:
import torch.nn as nn

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

# ConvMixer model with hard-coded parameters
def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10     # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [2]:
# Load the model
import torch

# Define the path to the model
device = "cuda" 

# Load the model
model = torch.load('/home/j597s263/scratch/j597s263/Models/ConvModels/Base/ConvMNIBase.mod', weights_only=False, map_location="cuda")
model = model.to(device)
model.eval()  

print("Model loaded successfully!")

Model loaded successfully!


In [3]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Subset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random

# Define dataset root directory
mnist_root = '/home/j597s263/scratch/j597s263/Datasets/MNIST'

random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root=mnist_root, transform=transform, train=True, download=True)
test_dataset = datasets.MNIST(root=mnist_root, transform=transform, train=False, download=True)

train_indices = list(range(len(train_dataset)))
random.shuffle(train_indices)  

split_idx = int(0.9 * len(train_indices))  
train_indices, attack_indices = train_indices[:split_idx], train_indices[split_idx:]

train_data = Subset(train_dataset, train_indices)
attack_data = Subset(train_dataset, attack_indices)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)  # Shuffle within batches
attack_loader = DataLoader(attack_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Total training samples: {len(train_dataset)}")
print(f"Training samples after split: {len(train_data)}")
print(f"Attack samples: {len(attack_data)}")
print(f"Testing samples: {len(test_dataset)}")

Total training samples: 60000
Training samples after split: 54000
Attack samples: 6000
Testing samples: 10000


In [4]:
ls /home/j597s263/scratch/j597s263/Datasets/Explanation_values/Conv/IG_ConvMNI

explanation_0.npy     explanation_2800.npy  explanation_4600.npy
explanation_1000.npy  explanation_2801.npy  explanation_4601.npy
explanation_1001.npy  explanation_2802.npy  explanation_4602.npy
explanation_1002.npy  explanation_2803.npy  explanation_4603.npy
explanation_1003.npy  explanation_2804.npy  explanation_4604.npy
explanation_1004.npy  explanation_2805.npy  explanation_4605.npy
explanation_1005.npy  explanation_2806.npy  explanation_4606.npy
explanation_1006.npy  explanation_2807.npy  explanation_4607.npy
explanation_1007.npy  explanation_2808.npy  explanation_4608.npy
explanation_1008.npy  explanation_2809.npy  explanation_4609.npy
explanation_1009.npy  explanation_280.npy   explanation_460.npy
explanation_100.npy   explanation_2810.npy  explanation_4610.npy
explanation_1010.npy  explanation_2811.npy  explanation_4611.npy
explanation_1011.npy  explanation_2812.npy  explanation_4612.npy
explanation_1012.npy  explanation_2813.npy  explanation_4613.npy
explanation_1013.npy  expl

In [5]:
import os
import numpy as np
import torch

# Initialize the aggregated array for MNIST (single channel)
aggregated_explanations = np.zeros((224, 224), dtype=np.float32)

# Define the directory containing the explanations
explanations_dir = "/home/j597s263/scratch/j597s263/Datasets/Explanation_values/Conv/IG_ConvMNI"

# Iterate through the attack loader to align images and their explanations
for idx, (images, labels) in enumerate(attack_loader):
    images, labels = images.to(device), labels.to(device)

    # Forward pass
    outputs = model(images)
    predicted_labels = outputs.argmax(dim=1).tolist()  # Convert tensor to list
    true_labels = labels.tolist()  # Convert tensor to list

    # Load explanation file
    explanation_file = os.path.join(explanations_dir, f"explanation_{idx}.npy")
    explanation_with_label = np.load(explanation_file)  # Shape expected: (1, 224, 224) for MNIST

    # Ensure explanation is grayscale (1-channel)
    explanation = explanation_with_label[0]  # Shape: (224, 224) after removing channel

    # Accumulate explanations
    aggregated_explanations += explanation  # Direct sum for MNIST (no RGB channels)

    print(f"Processed image {idx + 1}/{len(attack_loader)}")

print(aggregated_explanations)

Processed image 1/94
Processed image 2/94
Processed image 3/94
Processed image 4/94
Processed image 5/94
Processed image 6/94
Processed image 7/94
Processed image 8/94
Processed image 9/94
Processed image 10/94
Processed image 11/94
Processed image 12/94
Processed image 13/94
Processed image 14/94
Processed image 15/94
Processed image 16/94
Processed image 17/94
Processed image 18/94
Processed image 19/94
Processed image 20/94
Processed image 21/94
Processed image 22/94
Processed image 23/94
Processed image 24/94
Processed image 25/94
Processed image 26/94
Processed image 27/94
Processed image 28/94
Processed image 29/94
Processed image 30/94
Processed image 31/94
Processed image 32/94
Processed image 33/94
Processed image 34/94
Processed image 35/94
Processed image 36/94
Processed image 37/94
Processed image 38/94
Processed image 39/94
Processed image 40/94
Processed image 41/94
Processed image 42/94
Processed image 43/94
Processed image 44/94
Processed image 45/94
Processed image 46/

In [6]:
flattened_indices = aggregated_explanations.flatten().argsort()[-22:][::-1]  # Indices of top 22 values

top_22_coords = np.unravel_index(flattened_indices, aggregated_explanations.shape)
top_22_coords = list(zip(top_22_coords[0], top_22_coords[1]))

top_22_values = [aggregated_explanations[x, y] for x, y in top_22_coords]

top_22_pixels = list(zip(top_22_coords, top_22_values))

# Print the results
print("Top 22 Pixel Locations and Values:")
for coord, value in top_22_pixels:
    print(f"Pixel {coord}: Value {value:.4f}")

Top 22 Pixel Locations and Values:
Pixel (np.int64(0), np.int64(39)): Value 417.0000
Pixel (np.int64(0), np.int64(38)): Value 417.0000
Pixel (np.int64(0), np.int64(37)): Value 417.0000
Pixel (np.int64(0), np.int64(36)): Value 417.0000
Pixel (np.int64(0), np.int64(35)): Value 417.0000
Pixel (np.int64(0), np.int64(34)): Value 417.0000
Pixel (np.int64(0), np.int64(33)): Value 417.0000
Pixel (np.int64(0), np.int64(32)): Value 417.0000
Pixel (np.int64(0), np.int64(47)): Value 417.0000
Pixel (np.int64(0), np.int64(46)): Value 417.0000
Pixel (np.int64(0), np.int64(45)): Value 417.0000
Pixel (np.int64(0), np.int64(44)): Value 417.0000
Pixel (np.int64(0), np.int64(43)): Value 417.0000
Pixel (np.int64(0), np.int64(42)): Value 417.0000
Pixel (np.int64(0), np.int64(41)): Value 417.0000
Pixel (np.int64(0), np.int64(40)): Value 417.0000
Pixel (np.int64(0), np.int64(55)): Value 417.0000
Pixel (np.int64(0), np.int64(54)): Value 417.0000
Pixel (np.int64(0), np.int64(53)): Value 417.0000
Pixel (np.int64

In [7]:
import os
import numpy as np
from PIL import Image
import torch
from torchvision.transforms import ToPILImage

top_22_coords = [
    (0, 39), (0, 38), (0, 37), (0, 36), (0, 35),
    (0, 34), (0, 33), (0, 32), (0, 47), (0, 46),
    (0, 45), (0, 44), (0, 43), (0, 42), (0, 41),
    (0, 40), (0, 55), (0, 54), (0, 53), (0, 52),
    (0, 51), (0, 50)
]

save_dir = "/home/j597s263/scratch/j597s263/Datasets/Attack/ConvIGMni"  
os.makedirs(save_dir, exist_ok=True)

for idx, (images, labels) in enumerate(attack_loader):
    image = images[0].permute(1, 2, 0).cpu().numpy()  # Shape: (H, W, C)

    for x, y in top_22_coords:
        image[x, y] = [0, 0, 0]

    modified_image_tensor = torch.tensor(image).permute(2, 0, 1)  # Convert back to (C, H, W)
    pil_image = ToPILImage()(modified_image_tensor)

    save_path = os.path.join(save_dir, f"modified_image_{idx}.png")
    pil_image.save(save_path)

    print(f"Saved modified image {idx + 1}/{len(attack_loader)}")

print(f"All modified images saved to {save_dir}")

ValueError: could not broadcast input array from shape (3,) into shape (1,)